In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import re
import pandas as pd

In [3]:
driver = webdriver.Chrome()
url = "https://cafe.naver.com/imsanbu"
driver.get(url)
# 로그인!!!

In [52]:
# DataFrame 담을 리스트
pre_list = []
# 게시판 선택
for k in range(818, 828):
    source = driver.page_source
    soup = bs(source, 'lxml')
    element = driver.find_element_by_css_selector("#menuLink{}".format(k))
    time.sleep(0.8)
    element.click()
    
    # 게시글 번호 저장
    board_list = []
    title = []
    content = []
    for i in range(1,101):
        url = "https://cafe.naver.com/imsanbu?iframe_url=/ArticleList.nhn%3Fsearch.clubid=10094499%26search.menuid={}%26search.boardtype=L%26search.totalCount=151%26search.page={}".format(k, i)
        driver.get(url)
        time.sleep(0.8)
        driver.switch_to.frame("cafe_main")
        time.sleep(0.8)
        source = driver.page_source
        soup = bs(source, 'lxml')
        for j in soup.select('.board-number'):
            board_list.append(j.text)
        driver.switch_to.parent_frame()
        
    for h in board_list:
        try:
            target_url = "https://cafe.naver.com/imsanbu?iframe_url_utf8=%2FArticleRead.nhn%253Fclubid%3D10094499%2526page%3D10%2526menuid%3D818%2526boardtype%3DL%2526articleid%3D{}%2526referrerAllArticles%3Dfalse".format(h)
            driver.get(target_url)
            time.sleep(0.8)
            driver.switch_to.frame("cafe_main")
            time.sleep(0.8)
            source = driver.page_source
            soup = bs(source, 'lxml')
            title.append(soup.select('.title_text')[0].text.strip())
            try:
                content.append(soup.select('.se-component')[0].text.strip())
            except:
                content.append("none")
            driver.switch_to.parent_frame()
        except:
                pass

    result = {"제목" : title,'내용' : content}
    pre_list.append(pd.DataFrame(result))

In [54]:
# 주차별 확인 0~12
pre_list[10]

IndexError: list index out of range

In [55]:
# 최종 출력
for i in range(len(pre_list)):
    pre_list[i].to_csv("result"+str(i)+".csv", encoding="utf-8-sig")

In [ ]:
# # 초기 mam_df
# title = []
# content = []
# result = {"제목" : title, "내용" : content}
# mam_df = pd.DataFrame(result)

In [ ]:
# # mam_df에 합치기
# result1 = {"제목" : title,'내용' : content}
# df1 = pd.DataFrame(result1)
# mam_df = pd.concat([mam_df,df1])